In [ ]:
DATASET_URL = "http://mayorx.xyz/Media/agentifai/podcast_dataset.zip"
DATASET_PATH = "podcast_dataset.zip"
DATASET_DIR = "dataset/"
DATASET_NAME = "bitalk_podcast"
DATASET_LANG = "pt"


INPUT_VOICE_URL = "http://mayorx.xyz/Media/agentifai/input_voice.wav"
INPUT_VOICE_PATH = "input/voices/input_voice.wav"
INPUT_VOICE_DIR = "input/voices/"

MODEL_LATEST_URL = "http://mayorx.xyz/Media/agentifai/model_latest.pth"
MODEL_LATEST_VOCAB = "https://huggingface.co/coqui/XTTS-v2/resolve/main/vocab.json"
MODEL_LATEST_MEL = "https://huggingface.co/coqui/XTTS-v2/resolve/main/mel_stats.pth"
MODEL_LATEST_DVAE = "https://huggingface.co/coqui/XTTS-v2/resolve/main/dvae.pth"
MODEL_LATEST_MODEL_PATH = "run/training/XTTS_v2.0_original_model_files/model.pth"
MODEL_LATEST_VOCAB_PATH = "run/training/XTTS_v2.0_original_model_files/vocab.json"
MODEL_LATEST_MEL_PATH = "run/training/XTTS_v2.0_original_model_files/mel_stats.pth"
MODEL_LATEST_DVAE_PATH = "run/training/XTTS_v2.0_original_model_files/dvae.pth"
MODEL_LATEST_DIR = "run/training/XTTS_v2.0_original_model_files"

MODEL_BASE_URL = "https://huggingface.co/coqui/XTTS-v2/resolve/main/model.pth"

In [ ]:
# Get dataset single_speaker
!mkdir -p $INPUT_VOICE_DIR $MODEL_LATEST_DIR

# Download dataset
!wget $DATASET_URL -O $DATASET_PATH
!unzip $DATASET_PATH
!rm $DATASET_PATH

# Download input voice
!wget $INPUT_VOICE_URL -O $INPUT_VOICE_PATH

# Download model to FineTune
!wget $MODEL_LATEST_URL -O $MODEL_LATEST_MODEL_PATH
!wget $MODEL_LATEST_VOCAB -O $MODEL_LATEST_VOCAB_PATH
!wget $MODEL_LATEST_MEL -O $MODEL_LATEST_MEL_PATH
!wget $MODEL_LATEST_DVAE_PATH -O $MODEL_LATEST_DVAE_PATH

In [ ]:
!apt install espeak-ng ffmpeg

In [ ]:
!rm -rf coqui-ai-TTS
!git clone https://github.com/idiap/coqui-ai-TTS
!pip install -e coqui-ai-TTS/.

In [ ]:
import sys
sys.path.append('coqui-ai-TTS')

import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

In [ ]:


# Logging parameters
RUN_NAME = "XTTS_FineTuned"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# Set here the path that the checkpoints will be saved. Default: ./run/training/
OUT_PATH = os.path.join("model", "training")

# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # for multi-gpu training please make it False
START_WITH_EVAL = True  # if True it will star with evaluation
BATCH_SIZE = 8  # set here the batch size
GRAD_ACUMM_STEPS = 48  # set here the grad accumulation steps
# Note: we recommend that BATCH_SIZE * GRAD_ACUMM_STEPS need to be at least 252 for more efficient training. You can increase/decrease BATCH_SIZE but then set GRAD_ACUMM_STEPS accordingly.

# Define here the dataset that you want to use for the fine-tuning on.
config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name=DATASET_NAME,
    path=DATASET_PATH,
    meta_file_train="dataset/metadata.csv",
    language=DATASET_LANG,
)

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]

# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


# DVAE files
DVAE_CHECKPOINT_LINK = MODEL_LATEST_DVAE
MEL_NORM_LINK = MODEL_LATEST_MEL

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = MODEL_LATEST_VOCAB

XTTS_CHECKPOINT_LINK = MODEL_LATEST_URL

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = MODEL_LATEST_MODEL_PATH  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )


# Training sentences generations
SPEAKER_REFERENCE = [
    INPUT_VOICE_PATH  # speaker reference to be used in training test sentences
]
LANGUAGE = config_dataset.language


def main():
    # init args and config
    model_args = GPTArgs(
        max_conditioning_length=441000,  # 20 secs
        min_conditioning_length=22050,  # 1 secs
        debug_loading_failures=False,
        max_wav_length=441000,  # ~11.6 seconds
        max_text_length=256,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    # define audio config
    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=22050)
    # training parameters config
    config = GPTTrainerConfig(
        epochs=1000,
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=32,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=100,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        # target_loss="loss",
        print_eval=False,
        use_phonemes=True,
        phoneme_language=DATASET_LANG,
        phonemizer="espeak-ng",
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="radam",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="MultiStepLR",
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
          {
              "text": "Olá",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Obrigado",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Como estás",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Até amanhã",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Vamos ao cinema",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Está um dia bonito hoje",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Preciso de comprar pão na padaria",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "O cão correu atrás da bola no parque",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Ela leu o livro todo numa tarde chuvosa",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "O professor explicou a lição com muita paciência",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Eles decidiram viajar pelo país durante as férias",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "O sol brilhava intensamente no céu sem nuvens",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "No inverno gostamos de beber chá quente junto à lareira",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "A cidade estava cheia de luzes e decorações de Natal",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Estudamos juntos na biblioteca para preparar o exame final",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "O concerto foi magnífico com músicos talentosos e melodias emocionantes",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Depois de uma longa caminhada descansaram à sombra de uma árvore",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Ela preparou uma surpresa especial para o aniversário do irmão",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Os cientistas descobriram uma nova espécie de planta na floresta tropical",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "A exposição de arte contemporânea atraiu muitos visitantes este ano",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Apesar da chuva decidiram continuar com o piquenique no parque",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Os atletas treinaram arduamente para se prepararem para a competição",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "A biblioteca oferece uma vasta coleção de livros e recursos digitais",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Durante a viagem aprenderam muito sobre as culturas locais e tradições",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Com determinação e esforço alcançou todos os seus objetivos profissionais",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "No fim de semana planearam visitar a aldeia histórica e explorar as ruínas antigas",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
      ],
    )

    # init the model from config
    model = GPTTrainer.init_from_config(config)

    # load training samples
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # init the trainer and 🚀
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()


if __name__ == "__main__":
    main()